## SBC (single binary classifier) reduction and classification

In [ ]:
import numpy as np
import pandas as pd

X = np.array([[1, 2], [3, 4], [5, 6], [7, 8]])
X = pd.DataFrame(X)
y = np.array([0, 1, 2, 3])
y = pd.Series(y)

- for $K$ classes
- find $K-1$ parallel hyperplanes.
- $s$ is the number of classes to the left / right of the hyperplane.
- for $s = K-1$,
- an hyperplane separates 2 sets of classes.
- each hyperplane is a binary classification problem, with the additional constraint of parallelism.
- all these problems have to be solved simultaneously in an augmented feature space
    - for $x$ in $R^{d}$, the augmented feature space is $R^{d+1}$
- for each $x$ create $(K-1)$ new points: $(x, 0)(x,h_{1})...(x,h_{K-2})$, where $h$ is a positive constant
- define a binary training set
    - $(x_1,0)$ belongs to $C_1$, $(x_2,0)..(x_K,0)$ belong to $C_2$
    - $(x_1,h_1)$ and $(x_2,h_1)$ belongs to $C_1$, $(x_3,h_1)..(x_K, h_1)$ belong to $C_2$
    - ...
    - $(x_1,h_{K-2})..(x_{K-1},h_{K-2})$ belong to $C_1$, $(x_K,h_{K-2})$ belongs to $C_{K-1}$

In [ ]:
def sbc_reduction(X, y, h=1):
    # num of classes
    K = len(np.unique(y))
    
    print("original num classes: ", K)
    print("original num observations: ", X.shape[0])
    
    # num of parallel hyperplanes to be created (and replicas)
    s = K-1
    
    # if class labels not integer, convert to integer
    if not np.issubdtype(y.dtype, np.integer):
        new_y = pd.Series(pd.factorize(y)[0])
        # show the mapping
        mapping = pd.Series(pd.factorize(y)[1], index=np.unique(new_y))
        print("mapping: ", mapping)
        y = new_y
    
    # for each point, create s replicas each with a new feature in [0, h, h*2, ... h*(s-1)]
    # the new label is a binary label
    new_X = []
    new_y = []
    for i in range(X.shape[0]): # for each point
        for j in range(s): # for each replica
            new_X.append(np.append(X.iloc[i].values, h*j))
            new_label = y.iloc[i] <= j
            new_y.append(new_label.astype(int))
    
    new_X = pd.DataFrame(new_X).reset_index(drop=True)
    new_y = pd.DataFrame(new_y).reset_index(drop=True)
    new_data = pd.concat([new_X, new_y], axis=1)
    # rename binary label column
    new_data.columns = list(new_X.columns) + ['binary_label']
    
    print("new num classes: ", len(np.unique(new_y)))
    print("new num observations: ", new_X.shape[0], " (original num observations *", s, ")")
    
    return new_X, new_y, new_data


new_X, new_y, new_data = sbc_reduction(X, y)
print(new_data)

apply a linear two class classifier

In [ ]:
from sklearn.linear_model import LogisticRegression

logistic = LogisticRegression()
logistic.solver = 'liblinear'
logistic.penalty = 'l1'
logistic.fit(new_X, new_y.values.ravel())  # convert y to 1d array
pred_sbc_y = logistic.predict(new_X)
print(pred_sbc_y)

to classify an example
- classify all its replicas
- get a sequence of $K-1$ labels
- from this sequence, infer the class of the original example following the rule:
    (for $K=3$)
    - if both are $C_1$, then the class is $C_1$
    - if one is $C_1$ and the other is $C_2$, then the class is $C_2$
    - if both are $C_2$, then the class is $C_3$

In [ ]:
def sbc_classif(og_y, pred_sbc_y):
    K = len(np.unique(og_y)) # num of classes
    s = K-1 # num of hyperplanes / replicas
    
    # get classification of all replicas of each point
    all_labels = [pred_sbc_y[i:i + s] for i in range(0, len(pred_sbc_y), s)]
    all_labels = np.array(all_labels)
    print(all_labels)
    
    # get the class of the point
    # if all replicas are 0, then the class is 0
    # if one is 1 and the rest are 0, then the class is 1
    # if two are 1 and the rest are 0, then the class is 2
    # ...
    # if all replicas are 1, then the class is K
    final_labels = np.argmax(all_labels, axis=1)    
    
    return final_labels


final_labels = sbc_classif(y, pred_sbc_y)
print(final_labels)